In [31]:
# Render our plots inline
%matplotlib inline

from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from pandas import json_normalize

In [5]:
server = 'localhost'
#server = 'hv-6066.idi.ntnu.no'

port = '8080'
base_url = 'http://' + server + ':' + port + '/'

In [22]:
# returns the list of concepts existing in the CBR system
def get_concepts():
    raw = pd.DataFrame(requests.get(base_url + 'concepts/').json()) 
    concepts = pd.DataFrame.from_records(raw).values.tolist()
    return concepts

In [23]:
get_concepts()

[['Car']]

In [45]:
def get_attributes(concept):
    raw = json_normalize(requests.get(base_url + 'concepts/' + concept + '/attributes').json()).T
    attributes = raw.reset_index()
    attributes.columns = ['name', 'type']
    return attributes

In [46]:
get_attributes(get_concepts()[0][0]) # returns a dataframe of attribute names and their type existing in the CBR system

,name,type
0,ZIP,IntegerDesc
1,Speed,IntegerDesc
2,Color,SymbolDesc
3,CCM,IntegerDesc
4,Car Code,IntegerDesc
5,Miles,IntegerDesc
6,Year,IntegerDesc
7,Price,FloatDesc
8,Doors,IntegerDesc
9,Model,SymbolDesc


In [62]:
def retrieve_k_sim_byID_content(concept, casebase, amalFct, queryID, k):
    raw = pd.DataFrame(requests.get(base_url + 'concepts/' + concept + '/casebases/' 
                                        + casebase + '/amalgamationFunctions/'+amalFct+'/retrievalByCaseIDWithContent?caseID='+ queryID + '&k=' + k ).json()) 
    results = raw.apply(pd.to_numeric, errors='coerce').fillna(raw).sort_values(by='similarity', ascending=False)
    returned_df = results.apply(pd.to_numeric, errors='coerce').fillna(results)
    return returned_df

In [63]:
retrieve_k_sim_byID_content('Car', 'CaseBase0', 'CarFunc', '17_vw', '5')

,similarity,caseID,Body,CCM,Car Code,Color,Doors,Gas,Manufacturer,Miles,Model,Power,Price,Speed,Year,ZIP
0,1.0,17_vw,sedan,1900,17,dark_blue,4,diesel,vw,71433,passat,90,22099,183,1995,6
1,0.658371,891_vw,sedan,1900,891,blue,4,diesel,vw,68993,passat,110,23599,183,1995,2
2,0.658114,532_vw,sedan,1900,532,dark_blue,4,diesel,vw,13701,passat,90,33199,183,1996,3
3,0.632703,602_vw,sedan,1900,602,blue,4,gasoline,vw,80748,passat,193,26499,183,1996,1
4,0.632574,467_vw,sedan,1900,467,dark_blue,4,gasoline,vw,91911,passat,100,16699,183,1995,8
